In [1]:
%defaultDatasource jdbc:h2:mem:db

### Normalização
De acordo com a tabela principal, pensamos em modificações e otimizações reconhecendo as dependências funcionais entre as colunas. Percebemos que o país e o ano determinam o PIB anual, PIB per capita e a população. Analogamente, a idade determina a geração de um grupo de pessoas.
Logo, construímos as seguintes tabelas de acordo com a normalização feita:
 
Principal(**<ins>country</ins>**, **<ins>year</ins>**, **<ins>age</ins>**, **<ins>sex</ins>**, suicide_rate);
 
Pais(**<ins>country</ins>**, **<ins>year</ins>**, gdp_for_year, gdp_per_capita, population);
 
Geração(**<ins>age</ins>**, generation).
 
No entanto, a tabela tirada diretamente do Kaggle representa um modelo que não podemos usar diretamente para o nosso banco de dados relacional, logo é necessário a limpeza desses dados. A exemplo, temos a tabela de geração na qual grupos de idades determinam gerações diferentes.
 
Isso não foi feito durante esta etapa do trabalho mas estará feita até a próxima. Por isso, tivemos que considerar a criação excessiva de VIEWS para responder as perguntas que fizemos.
 
Os resultados das execuções das queries encontram-se abaixo.


# Importando Tabelas

In [2]:
DROP TABLE IF EXISTS Suicidios;
DROP TABLE IF EXISTS Geracao;
DROP TABLE IF EXISTS Pais;

CREATE TABLE  Suicidios(
    country VARCHAR(200),
    year INTEGER,
    sex VARCHAR(20),
    age VARCHAR(50),
    suicidio INTEGER,
    taxa_suicidio DOUBLE
) AS SELECT
    country,
    year,
    sex,
    age,
    suicides_no,
    suicides_rate
FROM CSVREAD('suicides.csv');

CREATE TABLE  Geracao(
    age VARCHAR(20),
    generation VARCHAR(20),
) AS SELECT
    age,
    generation
FROM CSVREAD('generation.csv');

CREATE TABLE  Pais(
    country VARCHAR(50),
    year INTEGER,
    gdp_for_year VARCHAR(50),
    gdp_per_capita  INTEGER,
    population INTEGER
) AS SELECT
    country,
    year,
    gdp_for_year,
    gdp_per_capita,
    population
FROM CSVREAD('pais.csv');


SELECT * FROM Suicidios;
SELECT * FROM Geracao;
SELECT * FROM Pais;

# Queries
1- Qual país teve maior taxa de suicídio em 2010?

Primeiro criamos uma view que agrupa as maiores de taxa de suicidio com o país no ano de 2010, em seguinte uma view que verifica a maior taxa de suícidio entre esses paises, assim a resposta é encontrada slecionar qual país possui essa taxa, neste caso é a Coreia do Sul.

In [3]:
DROP VIEW CMT IF EXISTS;
DROP VIEW PYT IF EXISTS;

CREATE VIEW PYT AS SELECT country, year, taxa_suicidio
    FROM Suicidios
    WHERE year = 2010
    GROUP BY country, year, taxa_suicidio;
 
CREATE VIEW CMT AS SELECT country, MAX(taxa_suicidio) taxa
    FROM PYT
    GROUP BY country;

SELECT country
    FROM (SELECT MAX(taxa) maior FROM CMT) AUX, CMT
    WHERE CMT.taxa = AUX.maior;

Republic of Korea

2 - Qual sexo tem a maior taxa de sucidios entre 25 e 34 anos em 2010?

Foi criado uma view que selecionna a taxa de suicídio de ambos os sexos entre 25 e 34 anos em 2010, após isso selecionado o maior, obtendo sexo masculino como resposta.

In [4]:
DROP VIEW ST IF EXISTS;

CREATE VIEW ST AS SELECT sex, MAX(taxa_suicidio) AS taxa
        FROM Suicidios
        WHERE age = '25-34 years' AND year = 2010
        GROUP BY sex;

SELECT ST.sex 
    FROM (SELECT MAX(taxa) maxima FROM ST) AS MAIOR, ST
    WHERE ST.taxa = MAIOR.maxima;

male

3 - Qual o PIB per capita do país com mais suicídios entre pessoas de 25 e 34 anos?

Foi criado vários views para limpar os dados da tabela e facilitar a busca, a primeira seleciona país, ano e taxa de suicidio entre pessoas de 25 a 34 anos, aí verficamos qual é a maior taxa e em que ano foi, e assim utilizamos a tabela Pais para descbobir o PIB per capita desse país e ano, resultando em 2853$.

In [5]:
DROP VIEW MY IF EXISTS;
DROP VIEW T1 IF EXISTS;
DROP VIEW CSS IF EXISTS;
DROP VIEW CAS IF EXISTS;

CREATE VIEW CAS AS SELECT country, age, suicidio
    FROM Suicidios
    WHERE age = '25-34 years'
    GROUP BY country, year, age, sex;


CREATE VIEW CSS AS SELECT country, SUM(suicidio) soma 
    FROM CAS 
    GROUP BY country 
    ORDER BY soma DESC;

CREATE VIEW T1 AS SELECT TOP 1 country, soma FROM CSS;

CREATE VIEW MY AS SELECT MAX(year) AS y
    FROM Pais
    WHERE country = 'Russian Federation';

SELECT P.gdp_per_capita
    FROM T1, MY, Pais P
    WHERE 'Russian Federation' = P.country AND P.year = MY.y;

9987

4 - Qual o PIB per capita do país com a menor taxa de suicídio entre os mais idosos(75+ anos) em 2015?


Semelhante a 3 usamos views para fazer uma limpeza nos dados, criamos uma  view com a  taxa de suicídio entre os idosos, pais e ano, verificamos a menor taxa de suicídio e com qual país e ano ela está relacionada, assim é possível selecionar qual o PIB desse país nesse ano, resultando em 7326$.

In [6]:
DROP VIEW MenoresTaxasEm2015 IF EXISTS;

CREATE VIEW MenoresTaxasEm2015 AS SELECT country, AVG(taxa_suicidio) media
FROM Suicidios
WHERE age = '75+ years' AND year = 2015
GROUP BY country, year
HAVING media > 0
ORDER BY media;

SELECT TOP 1 country FROM MenoresTaxasEm2015;

Turkmenistan

In [7]:
SELECT gdp_per_capita
FROM Pais
WHERE country = 'Turkmenistan' AND year = 2015

7326

5- Qual a variação, em porcentagem, do índice de suicídios do ano 2010 em relação ao ano 2007 no Brasil?

Foi pego a soma do indice de suicídio de pessoas de 25 a 34 no Brasil em 2007 e em 2010, com isso foi calculado a diferença percentual resultando num aumento de aproximadamente 11.74%.

In [8]:
DROP VIEW IF EXISTS SuicideNoBr2007;
DROP VIEW IF EXISTS SuicideNoBr2010;

CREATE VIEW SuicideNoBr2007 AS
SELECT CAST(SUM(S.suicidio) AS DOUBLE) suicidio
FROM Suicidios S
WHERE S.country = 'Brazil' AND S.year = 2007 AND S.age = '25-34 years';

CREATE VIEW SuicideNoBr2010 AS
SELECT CAST(SUM(S.suicidio) AS DOUBLE) suicidio
FROM Suicidios S
WHERE S.country = 'Brazil' AND S.year = 2010 AND S.age = '25-34 years';

SELECT ( ( (S10.suicidio/S07.suicidio) - 1)*100) 
FROM SuicideNoBr2007 S07, SuicideNoBr2010 S10;

11.74412793603199

## Queries para neo4j
Nesta seção, levantamos algumas queries para relacionar com alguns filmes renomados do IMBd.

De acordo com mudanças abruptas da taxa de suicídios, queremos verificar se houve lançamentos de filmes renomados e de cunho melancólico com objetivo de verificar o Efeito Werther.

Primeiro, selecionamos dois países para estudo: Estados Unidos e França. Segundo, vamos verificar a diferença entre médias em 3 períodos distintos e, terceiro, vamos fazer uma rápida comparação entre a taxa de suicídio mais atual dos Millenials com outras gerações.

### Estado Unidos

In [9]:
SELECT MIN(year), MAX(year) 
FROM Suicidios 
WHERE country = 'United States'
GROUP BY country;

In [10]:
SELECT AVG(taxa_suicidio)
FROM Suicidios
WHERE country = 'United States' AND year >= 1985 AND year < 1995
GROUP BY country;

SELECT AVG(taxa_suicidio)
FROM Suicidios 
WHERE country = 'United States' AND year >= 1995 AND year < 2005
GROUP BY country;

SELECT AVG(taxa_suicidio)
FROM Suicidios
WHERE country = 'United States' AND year >= 2005 AND year <= 2015
GROUP BY country;

### France

In [11]:
SELECT MIN(year), MAX(year) 
FROM Suicidios 
WHERE country = 'France'
GROUP BY country;

In [12]:
SELECT AVG(taxa_suicidio)
FROM Suicidios
WHERE country = 'France' AND year >= 1985 AND year < 1995
GROUP BY country;

SELECT AVG(taxa_suicidio)
FROM Suicidios 
WHERE country = 'France' AND year >= 1995 AND year < 2005
GROUP BY country;

SELECT AVG(taxa_suicidio)
FROM Suicidios
WHERE country = 'France' AND year >= 2005 AND year < 2015
GROUP BY country;

### Brazil

In [13]:
SELECT MIN(year), MAX(year) 
FROM Suicidios 
WHERE country = 'Brazil'
GROUP BY country;

In [14]:
SELECT AVG(taxa_suicidio)
FROM Suicidios
WHERE country = 'Brazil' AND year >= 1985 AND year < 1995
GROUP BY country;

SELECT AVG(taxa_suicidio)
FROM Suicidios 
WHERE country = 'Brazil' AND year >= 1995 AND year < 2005
GROUP BY country;

SELECT AVG(taxa_suicidio)
FROM Suicidios
WHERE country = 'Brazil' AND year >= 2005 AND year <= 2015
GROUP BY country;

### Maiores taxas entre gerações 
Primeiro: comparativo entre pessoas da faixa etária de 15 a 24 anos, todos os anos.
Segundo: comparativo entre pessoas da faixa etária de 15 a 24 anos, em 2014.

In [15]:
DROP VIEW MaioresTaxas IF EXISTS;
DROP VIEW MediaDosAnos IF EXISTS;

CREATE VIEW MediaDosAnos AS SELECT country, age, AVG(taxa_suicidio) media_taxa
FROM Suicidios
GROUP BY country, age
HAVING media_taxa > 0;

CREATE VIEW MaioresTaxas AS SELECT country, age, MAX(media_taxa)
FROM MediaDosAnos
GROUP BY country, age;

SELECT COUNT(age) FROM MaioresTaxas;
SELECT COUNT(age) FROM MaioresTaxas WHERE age = '15-24 years';

In [16]:
DROP VIEW MaioresTaxas IF EXISTS;
DROP VIEW MediaDosAnos IF EXISTS;

CREATE VIEW MediaDosAnos AS SELECT country, age, AVG(taxa_suicidio) media_taxa
FROM Suicidios
WHERE year = 2014
GROUP BY country, age
HAVING media_taxa > 0;

CREATE VIEW MaioresTaxas AS SELECT country, age, MAX(media_taxa)
FROM MediaDosAnos
GROUP BY country, age;

SELECT COUNT(age) FROM MaioresTaxas;
SELECT COUNT(age) FROM MaioresTaxas WHERE age = '15-24 years';